In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
from os import listdir,remove
from tensorflow   import keras
import matplotlib.pyplot as plt
from keras.utils import plot_model
from datetime import datetime as dt
from os.path import isfile,isdir,join,splitext,basename
from sklearn.model_selection import train_test_split as tts

In [0]:
def get_frame(video_source):
  video = cv2.VideoCapture(video_source)
  ret, frame = video.read()
  video.release()
  cv2.destroyAllWindows()
  return frame

def get_frame_count(video_source):
  video = cv2.VideoCapture(video_source)
  counter=0
  ret,frame=video.read()
  while ret:
    counter+=1
    ret,frame=video.read()
  video.release()
  cv2.destroyAllWindows()
  return counter

def get_sub_video(video_source,destiny_path,frames):
  if isfile(video_source):
    frame_count=get_frame_count(video_source)
    step=int(frame_count/frames)
    if step>0:
      video = cv2.VideoCapture(video_source)
      counter=0
      ret,frame=video.read()
      h,w,_=np.shape(frame)
      fps = video.get(cv2.CAP_PROP_FPS)
      fourcc = cv2.VideoWriter_fourcc(*'png ')
      dst=join(destiny_path,basename(video_source))
      if isfile(dst):
        remove(dst)
      videowriter = cv2.VideoWriter(dst,fourcc, fps, (w,h))
      rec_counter=0
      while ret:
        counter+=1
        if counter%step==0 and rec_counter<frames:
          rec_counter+=1
          videowriter.write(frame)
        ret,frame=video.read()
      videowriter.release()
      video.release()
      cv2.destroyAllWindows()
      return counter
    else:
      print("Video not processed: ",basename(video_source))
  else:
    print("source is not video:",video_source)

def all_patients(videos_path):
  patients=[]
  if isdir(videos_path):
    videos=listdir(videos_path)
    for video in videos:
      if [video.split("_")[1],video.split("_")[2]] not in patients:
        patients.append([video.split("_")[1],video.split("_")[2]])
  return patients

def leave_one_out(videos_path,patient=None):
  if isdir(videos_path):
    videos=listdir(videos_path)
    patients=all_patients(videos_path)
    if patient==None:#take random_patient as test
      patient=patients[random.randint(0,len(patients)-1)]
    elif len(patient)==2:
      patients.remove(patient)
    test_videos=[join(videos_path,video) for video in videos if patient[0]+"_"+patient[1] in video]
    aux=[[join(videos_path,video) for video in videos if patient[0]+"_"+patient[1] in video] for patient in patients]
    auxx=[]
    [auxx.extend(item) for item in aux]
    train_videos=[]
    [train_videos.append(item) for item in auxx if item not in train_videos]

    if len(test_videos)>0:
      train_videos.sort()
      test_videos.sort()
      return train_videos,test_videos

def train_test(videos_path,test_size):
  if isdir(videos_path):
    videos=listdir(videos_path)
    l=len(videos)/2
    n=int(np.round(l*test_size))
    m=int(l-n)
    patients=all_patients(videos_path)
    random.shuffle(patients)
    train_patients=patients[0:m]
    test_patients=patients[m:]
    aux=[[join(videos_path,video) for video in videos if patient[0]+"_"+patient[1] in video] for patient in train_patients]
    auxx=[]
    [auxx.extend(item) for item in aux]
    train_videos=[]
    [train_videos.append(item) for item in auxx if item not in train_videos]
    aux=[[join(videos_path,video) for video in videos if patient[0]+"_"+patient[1] in video] for patient in test_patients]
    auxx=[]
    [auxx.extend(item) for item in aux]
    test_videos=[]
    [test_videos.append(item) for item in auxx if item not in train_videos]
    return train_videos,test_videos


def videos_to_x(videos_sources,normalize=True):
  samples=len(videos_sources)
  depth=get_frame_count(videos_sources[0])
  h,w,c=np.shape(get_frame(videos_sources[0]))
  array=np.zeros(shape=[samples,depth,w,h,c])
  sample=0
  for video in videos_sources:
    video = cv2.VideoCapture(video)
    counter=0
    ret,frame=video.read()
    while ret:
      array[sample,counter,:,:,:]=frame
      counter+=1
      ret,frame=video.read()
    video.release()
    cv2.destroyAllWindows()
    sample+=1
  if normalize:
    array=array/255
  return array

def videos_to_y(videos_sources):
  y=[]
  for video in videos_sources:
    y.append(int(basename(video).split("_")[2]))
  y=np.asarray(y)
  return y

In [49]:
videos_path="/content/drive/My Drive/Proyecto_de_grado/aux_data/accelerated"
destiny_path="/content/drive/My Drive/Proyecto_de_grado/aux_data/summarized"
opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)
videos=[join(videos_path,file) for file in listdir(videos_path)]
print("Total dataset samples:",len(videos))
frames=80
res_src="/content/drive/My Drive/Proyecto_de_grado/results.txt"
#for video in videos:
#  get_sub_video(video,destiny_path,frames)

Total dataset samples: 52


In [0]:
model_CNN =  keras.models.Sequential()
model_CNN.add(keras.layers.Conv3D(8, (5, 5, 5), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(frames,150, 150, 3)))
model_CNN.add(keras.layers.BatchNormalization())
model_CNN.add(keras.layers.ReLU())
model_CNN.add(keras.layers.Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_CNN.add(keras.layers.BatchNormalization())
model_CNN.add(keras.layers.ReLU())
model_CNN.add(keras.layers.Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_CNN.add(keras.layers.BatchNormalization())
model_CNN.add(keras.layers.ReLU())
model_CNN.add(keras.layers.MaxPooling3D((2, 2, 2)))
model_CNN.add(keras.layers.Conv3D(64, (1, 1, 1), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_CNN.add(keras.layers.BatchNormalization())
model_CNN.add(keras.layers.ReLU())
model_CNN.add(keras.layers.Flatten())
model_CNN.add(keras.layers.Dense(10,activation='relu', kernel_initializer='he_uniform'))
model_CNN.add(keras.layers.Dense(2, activation='softmax'))

In [51]:
model_CNN.summary()
#plot_model(model_CNN, to_file='/content/drive/My Drive/Proyecto_de_grado/aux_data/model_3D.png')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_12 (Conv3D)           (None, 80, 150, 150, 8)   3008      
_________________________________________________________________
batch_normalization_12 (Batc (None, 80, 150, 150, 8)   32        
_________________________________________________________________
re_lu_12 (ReLU)              (None, 80, 150, 150, 8)   0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 80, 150, 150, 16)  3472      
_________________________________________________________________
batch_normalization_13 (Batc (None, 80, 150, 150, 16)  64        
_________________________________________________________________
re_lu_13 (ReLU)              (None, 80, 150, 150, 16)  0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 80, 150, 150, 32) 

In [52]:
"""
already=[]
lines=open(res_src,"r").read().split("\n")
for line in lines:
  if len(line)>0:
    aux=line.split(";")
    already.append([aux[0],aux[1]])
patients=all_patients(destiny_path)
for patient in patients:
  print("Patient:",patient)
  if patient not in already:
    print("  Process:",patient)
    train,test=leave_one_out(destiny_path,patient=patient)
    x_train=videos_to_x(train)
    y_train=videos_to_y(train)
    x_test=videos_to_x(test)
    y_test=videos_to_y(test)
    model_CNN.compile(optimizer=opt, loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    model_CNN.fit(x_train, y_train, epochs=8, batch_size=10)
    y_pred=model_CNN.predict(x_test)
    f=open(res_src,"a+")
    text=patient[0]+";"+patient[1]+";"+str(y_test[0])+";"+str(y_test[1])+";"+str(np.argmax(y_pred[0]))+";"+str(np.argmax(y_pred[1]))+"\n"
    f.write(text)
    f.close()
"""

'\nalready=[]\nlines=open(res_src,"r").read().split("\n")\nfor line in lines:\n  if len(line)>0:\n    aux=line.split(";")\n    already.append([aux[0],aux[1]])\npatients=all_patients(destiny_path)\nfor patient in patients:\n  print("Patient:",patient)\n  if patient not in already:\n    print("  Process:",patient)\n    train,test=leave_one_out(destiny_path,patient=patient)\n    x_train=videos_to_x(train)\n    y_train=videos_to_y(train)\n    x_test=videos_to_x(test)\n    y_test=videos_to_y(test)\n    model_CNN.compile(optimizer=opt, loss=\'sparse_categorical_crossentropy\',metrics=[\'accuracy\'])\n    model_CNN.fit(x_train, y_train, epochs=8, batch_size=10)\n    y_pred=model_CNN.predict(x_test)\n    f=open(res_src,"a+")\n    text=patient[0]+";"+patient[1]+";"+str(y_test[0])+";"+str(y_test[1])+";"+str(np.argmax(y_pred[0]))+";"+str(np.argmax(y_pred[1]))+"\n"\n    f.write(text)\n    f.close()\n'

In [0]:
print(len(listdir(destiny_path)))
train,test=train_test(destiny_path,0.2)
x_train=videos_to_x(train)
y_train=videos_to_y(train)
x_test=videos_to_x(test)
y_test=videos_to_y(test)
model_CNN.compile(optimizer=opt, loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model_CNN.fit(x_train, y_train, epochs=8, batch_size=10,validation_data=(x_test, y_test))

52
Epoch 1/8
5/5 [==============================] - 415s 83s/step - loss: 150.0954 - accuracy: 0.4762 - val_loss: 0.6933 - val_accuracy: 0.4000
Epoch 2/8
5/5 [==============================] - 416s 83s/step - loss: 0.6931 - accuracy: 0.5238 - val_loss: 0.6933 - val_accuracy: 0.4000
Epoch 3/8
5/5 [==============================] - 416s 83s/step - loss: 0.6931 - accuracy: 0.5238 - val_loss: 0.6934 - val_accuracy: 0.4000
Epoch 4/8
5/5 [==============================] - 414s 83s/step - loss: 0.6931 - accuracy: 0.5238 - val_loss: 0.6934 - val_accuracy: 0.4000
Epoch 5/8
1/5 [=====>........................] - ETA: 0s - loss: 0.6926 - accuracy: 0.7000

In [0]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy del modelo')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Pérdida del modelo')
plt.ylabel('pérdida')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()